In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('../Data/train.csv')
df.columns = [x.title().strip() for x in df.columns]
df = df.dropna(subset=['S1'])

drop_cols = ['S1_Large', 'S2_Large', 'S3_Large']

In [3]:
df['Event'].value_counts()

for index, row in df.iterrows():
    number = str(row['Number'])
    location_number = row['Location'][-1:]
    if row['Event'] == 'Free Practice 1':
        event = 'FP1'
    elif row['Event'] == 'Free Practice 2':
        event = 'FP2'
    elif row['Event'] == 'Free Practice 3':
        event = 'FP3'
    elif row['Event'] == 'Qualifying Group 1':
        event = 'QG1'
    elif row['Event'] == 'Qualifying Group 2':
        event = 'QG2'
    elif row['Event'] == 'Qualifying Group 3':
        event = 'QG3'
    elif row['Event'] == 'Qualifying Group 4':
        event = 'QG4'
    df.at[index, 'Trial_ID'] = event + '-' + location_number + '-' + number

In [4]:
for x in df['Trial_ID'].unique():
    temp = df.loc[df['Trial_ID'] == x]
    
    lap_number_previous = 1
    trial_identifier = 1
    
    for index, row in temp.iterrows():
        if row['Lap_Number'] >= lap_number_previous:
            df.at[index, 'Trial_Number'] = trial_identifier
            lap_number_previous += 1
        elif row['Lap_Number'] < lap_number_previous:
            trial_identifier += 1
            df.at[index, 'Trial_Number'] = trial_identifier
            lap_number_previous = 1
            
df['Trial_ID_2'] = df['Trial_ID'] + '-' + df['Trial_Number'].astype(int).astype(str)

In [5]:
def TimeConversion(x):
    x = str(x)
    if x != 'nan':
        try:
            y = datetime.strptime(x, '%M:%S.%f').time()
        except ValueError:
            try:
                y = datetime.strptime(x, '%S.%f').time()
            except ValueError:
                try:
                    y = datetime.strptime(x, '%S').time()
                except ValueError:
                    y = datetime.strptime('0', '%S').time()
    if x == 'nan':
        y = datetime.strptime('0', '%S').time()
    z = timedelta(minutes=y.minute, seconds=y.second, microseconds=y.microsecond)
    return z

In [6]:
time_cols = [
    'S1',
    'S2',
    'S3',
    'Elapsed',
    'Hour',
    'S1_Large',
    'S2_Large',
    'S3_Large',
    'Pit_Time',
]

daytime_cols = ['Hour']

for x in time_cols:
    df[x] = df[x].apply(TimeConversion)

In [7]:
for x in df['Trial_ID_2']:
    temp = df.loc[df['Trial_ID_2'] == x]
    laps = len(temp)
    pit_time = timedelta(0)
    for index, row in temp.iterrows():
        if ~pd.isna(row['Pit_Time']):
            pit_amount = row['Pit_Time']
            pit_time += pit_amount
            
    df.loc[df['Trial_ID_2'] == x, 'Pit_Time'] = pit_time / laps

In [8]:
test = df[df['Trial_ID_2'] == 'FP2-2-10-1']
test

,Number,Driver_Number,Lap_Number,Lap_Time,Lap_Improvement,Crossing_Finish_Line_In_Pit,S1,S1_Improvement,S2,S2_Improvement,S3,S3_Improvement,Kph,Elapsed,Hour,S1_Large,S2_Large,S3_Large,Driver_Name,Pit_Time,Group,Team,Power,Location,Event,Trial_ID,Trial_Number,Trial_ID_2
0,10,1,1,92,0,NaN,00:05:43.300000,0,00:00:35.427000,0,00:00:43.313000,0,28.8,00:07:02,00:22:02,00:05:43.300000,00:00:35.400000,00:00:43.300000,SB,00:01:44.900000,NaN,JR,NaN,Location 2,Free Practice 2,FP2-2-10,1.0,FP2-2-10-1
1,10,1,2,87,2,NaN,00:00:25.674000,2,00:00:33.399000,2,00:00:41.922000,2,120.5,00:08:43,00:23:43,00:00:25.700000,00:00:33.400000,00:00:41.900000,SB,00:01:44.900000,NaN,JR,NaN,Location 2,Free Practice 2,FP2-2-10,1.0,FP2-2-10-1
2,10,1,3,73,0,B,00:00:28.129000,0,00:00:34.091000,0,00:00:57.248000,0,101.9,00:10:42.500000,00:25:42.500000,00:00:28.100000,00:00:34.100000,00:00:57.200000,SB,00:01:44.900000,NaN,JR,NaN,Location 2,Free Practice 2,FP2-2-10,1.0,FP2-2-10-1


In [9]:
df = df.drop(columns=[
    'S1_Large',
    'S2_Large',
    'S3_Large',
    'Trial_Number',
    'Trial_ID',
    'Number',
    'Driver_Number',
    'Crossing_Finish_Line_In_Pit'
])

In [10]:
df

,Lap_Number,Lap_Time,Lap_Improvement,S1,S1_Improvement,S2,S2_Improvement,S3,S3_Improvement,Kph,Elapsed,Hour,Driver_Name,Pit_Time,Group,Team,Power,Location,Event,Trial_ID_2
0,1,92,0,00:05:43.300000,0,00:00:35.427000,0,00:00:43.313000,0,28.8,00:07:02,00:22:02,SB,00:01:44.900000,NaN,JR,NaN,Location 2,Free Practice 2,FP2-2-10-1
1,2,87,2,00:00:25.674000,2,00:00:33.399000,2,00:00:41.922000,2,120.5,00:08:43,00:23:43,SB,00:01:44.900000,NaN,JR,NaN,Location 2,Free Practice 2,FP2-2-10-1
2,3,73,0,00:00:28.129000,0,00:00:34.091000,0,00:00:57.248000,0,101.9,00:10:42.500000,00:25:42.500000,SB,00:01:44.900000,NaN,JR,NaN,Location 2,Free Practice 2,FP2-2-10-1
3,1,73,0,00:01:05,0,00:00:38.416000,0,00:00:56.833000,0,75.9,00:02:40.200000,00:17:40.200000,LGRA,00:00:08.250000,NaN,AD,NaN,Location 2,Free Practice 2,FP2-2-11-1
4,2,73,0,00:00:28.013000,0,00:00:36.743000,0,00:00:44.716000,0,111.2,00:04:29.700000,00:19:29.700000,LGRA,00:00:08.250000,NaN,AD,NaN,Location 2,Free Practice 2,FP2-2-11-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10271,17,84,0,00:00:22.936000,0,00:00:21.231000,0,00:00:23.701000,0,124.9,00:23:54.700000,00:23:54.700000,PWEHRL,00:00:12.580952,NaN,TAG,250.0,Location 8,Free Practice 1,FP1-8-99-2
10272,18,70,0,00:00:23.610000,0,00:00:22.432000,0,00:00:30.281000,0,111.1,00:25:11,00:25:11,PWEHRL,00:00:12.580952,NaN,TAG,235.0,Location 8,Free Practice 1,FP1-8-99-2
10273,19,70,0,00:02:17.100000,0,00:00:22.681000,0,00:00:24.308000,0,46.1,00:28:15.100000,00:28:15.100000,PWEHRL,00:00:12.580952,NaN,TAG,250.0,Location 8,Free Practice 1,FP1-8-99-2
10274,20,155,2,00:00:22.539000,2,00:00:21.057000,2,00:00:23.548000,2,126.3,00:29:22.300000,00:29:22.300000,PWEHRL,00:00:12.580952,NaN,TAG,250.0,Location 8,Free Practice 1,FP1-8-99-2


## Function

In [2]:
def DataProcessing(csv_url):
    df = pd.read_csv(csv_url)
    df.columns = [x.title().strip() for x in df.columns]
    df = df.dropna(subset=['S1'])

    drop_cols = ['S1_Large', 'S2_Large', 'S3_Large']

    df['Event'].value_counts()

    for index, row in df.iterrows():
        number = str(row['Number'])
        location_number = row['Location'][-1:]
        if row['Event'] == 'Free Practice 1':
            event = 'FP1'
        elif row['Event'] == 'Free Practice 2':
            event = 'FP2'
        elif row['Event'] == 'Free Practice 3':
            event = 'FP3'
        elif row['Event'] == 'Qualifying Group 1':
            event = 'QG1'
        elif row['Event'] == 'Qualifying Group 2':
            event = 'QG2'
        elif row['Event'] == 'Qualifying Group 3':
            event = 'QG3'
        elif row['Event'] == 'Qualifying Group 4':
            event = 'QG4'
        df.at[index, 'Trial_ID'] = event + '-' + location_number + '-' + number

    for x in df['Trial_ID'].unique():
        temp = df.loc[df['Trial_ID'] == x]

        lap_number_previous = 1
        trial_identifier = 1

        for index, row in temp.iterrows():
            if row['Lap_Number'] >= lap_number_previous:
                df.at[index, 'Trial_Number'] = trial_identifier
                lap_number_previous += 1
            elif row['Lap_Number'] < lap_number_previous:
                trial_identifier += 1
                df.at[index, 'Trial_Number'] = trial_identifier
                lap_number_previous = 1

    df['Trial_ID_2'] = df['Trial_ID'] + '-' + df['Trial_Number'].astype(int).astype(str)

    def TimeConversion(x):
        x = str(x)
        if x != 'nan':
            try:
                y = datetime.strptime(x, '%M:%S.%f').time()
            except ValueError:
                try:
                    y = datetime.strptime(x, '%S.%f').time()
                except ValueError:
                    try:
                        y = datetime.strptime(x, '%S').time()
                    except ValueError:
                        y = datetime.strptime('0', '%S').time()
        if x == 'nan':
            y = datetime.strptime('0', '%S').time()
        z = timedelta(minutes=y.minute, seconds=y.second, microseconds=y.microsecond)
        return z

    time_cols = [
        'S1',
        'S2',
        'S3',
        'Elapsed',
        'Hour',
        'S1_Large',
        'S2_Large',
        'S3_Large',
        'Pit_Time',
    ]

    daytime_cols = ['Hour']

    for x in time_cols:
        df[x] = df[x].apply(TimeConversion)

    for x in df['Trial_ID_2']:
        temp = df.loc[df['Trial_ID_2'] == x]
        laps = len(temp)
        pit_time = timedelta(0)
        for index, row in temp.iterrows():
            if ~pd.isna(row['Pit_Time']):
                pit_amount = row['Pit_Time']
                pit_time += pit_amount

        df.loc[df['Trial_ID_2'] == x, 'Pit_Time'] = pit_time / laps
        
    return df